In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

# Parametry pocisku
m = 0.004  # masa pocisku w kilogramach (4 gramy)
d = 0.00556       # średnica w metrach
A = np.pi * (d / 2) ** 2  # pole powierzchni czołowej
Cd = 0.295        # współczynnik oporu
rho0 = 1.225      # gęstość powietrza na poziomie morza (kg/m^3)
H = 8500          # wysokość skali atmosferycznej w metrach
g = 9.81          # przyspieszenie ziemskie (m/s^2)
v0 = 890          # prędkość początkowa w m/s (dla Grot)

def rho(h):
    return rho0 * np.exp(-h / H)

def vertical_motion_with_variable_drag(t, y):
    v, h = y
    if h < 0:
        return [0, 0]
    air_density = rho(h)
    drag_force = 0.5 * air_density * Cd * A * v**2 * np.sign(v)
    dvdt = -g - drag_force / m
    dhdt = v
    return [dvdt, dhdt]

y0 = [v0, 0.001]

t_span = (0, 45)
t_eval = np.linspace(t_span[0], t_span[1], 1000)
sol_up = solve_ivp(vertical_motion_with_variable_drag, t_span, y0, t_eval=t_eval, rtol=1e-8)

# Kinetic energy at all points
KEs = 0.5 * m * sol_up.y[0]**2

# Szukanie maksimum wysokości i czasu osiągnięcia
max_height = np.max(sol_up.y[1])
apex_idx = np.argmax(sol_up.y[1])
time_to_max_height = sol_up.t[apex_idx]

# Szukanie lądowania (h = 0, interpolacja)
h_vals = sol_up.y[1]
land_idx = np.where(h_vals <= 0)[0]
if len(land_idx) > 0:
    idx0 = land_idx[0]
    if idx0 == 0:
        landing_time = sol_up.t[0]
        landing_height = 0.0
        landing_ke = KEs[0]
    else:
        # interpolacja liniowa
        t1, t2 = sol_up.t[idx0-1], sol_up.t[idx0]
        h1, h2 = h_vals[idx0-1], h_vals[idx0]
        v1, v2 = sol_up.y[0][idx0-1], sol_up.y[0][idx0]
        # interpolacja czasu i prędkości dla h=0
        landing_time = t1 + (0 - h1) * (t2 - t1) / (h2 - h1)
        landing_v = v1 + (0 - h1) * (v2 - v1) / (h2 - h1)
        landing_height = 0.0
        landing_ke = 0.5 * m * landing_v**2
else:
    landing_time = sol_up.t[-1]
    landing_height = sol_up.y[1][-1]
    landing_ke = KEs[-1]

# Wykres
plt.figure(figsize=(10, 5))
plt.plot(sol_up.t, sol_up.y[1], color='mediumturquoise', label='Wysokość pocisku')

plt.xlabel("Czas (s)")
plt.ylabel("Wysokość pocisku (m)")
plt.title("Wznoszenie i opadanie pocisku z karabinka GROT\nz uwzględnieniem oporu powietrza i zmiennej gęstości", fontsize=12)

plt.annotate(f'Szczyt lotu:\nenergia kinetyczna: {KEs[apex_idx]:.1f} J',
             xy=(time_to_max_height, max_height),
             xytext=(time_to_max_height + 1, max_height - 300),
             arrowprops=dict(arrowstyle='->', color='black'),
             bbox=dict(boxstyle='round,pad=0.2', fc='lightyellow', ec='gray', lw=1),
             fontsize=10)

plt.annotate(f'Start lotu:\nenergia kinetyczna: {KEs[0]:.1f} J',
             xy=(0, 0),
             xytext=(2, 200),
             arrowprops=dict(arrowstyle='->', color='black'),
             bbox=dict(boxstyle='round,pad=0.2', fc='lightyellow', ec='gray', lw=1),
             fontsize=10)

plt.annotate(f'Koniec lotu:\nenergia kinetyczna: {landing_ke:.1f} J',
             xy=(landing_time, landing_height),
             xytext=(landing_time - 7, 500),
             arrowprops=dict(arrowstyle='->', color='black'),
             bbox=dict(boxstyle='round,pad=0.2', fc='lightyellow', ec='gray', lw=1),
             fontsize=10)

plt.legend()
plt.show()